# Imports

In [1]:
import tensorflow as tf

import os
import tempfile
import zipfile
import tqdm
import glob

from cral.pipeline import ObjectDetectionPipe
from cral.models.object_detection import RetinanetConfig


# Download Chess dataset

In [2]:
def download_chess_dataset(dataset_path=tempfile.gettempdir()):
    zip_url = 'https://public.roboflow.ai/ds/uBYkFHtqpy?key=HZljsh2sXY'
    path_to_zip_file = tf.keras.utils.get_file(
        'chess_pieces.zip',
        zip_url,
        cache_dir=dataset_path, 
        cache_subdir='',
        extract=False)
    directory_to_extract_to = os.path.join(dataset_path,'chess_pieces')
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)

    images_dir = os.path.join(dataset_path, 'chess_pieces','train')
    annotation_dir = os.path.join(dataset_path, 'chess_pieces','train')

    for image in tqdm.tqdm(glob.glob(os.path.join(images_dir, '*.jpg'))):
        new_name = image.replace('_jpg.rf.', '')
        os.rename(image, new_name)

        annotation = image.replace('.jpg', '.xml')
        new_name = annotation.replace('_jpg.rf.', '')
        os.rename(annotation, new_name)

    return images_dir, annotation_dir

In [3]:
images_dir, annotation_dir = download_chess_dataset('./')

8060928/8055750 [==============================] - 0s 0us/step


100%|██████████| 202/202 [00:00<00:00, 31012.06it/s]


# SetUp pipeline

In [4]:
pipe = ObjectDetectionPipe()

pipe.add_data(
    train_images_dir=images_dir,
    train_anno_dir=annotation_dir,
    annotation_format='pascal_voc',
    split=0.2)

pipe.lock_data()

pipe.set_algo(feature_extractor='resnet50', config=RetinanetConfig())

  0%|                                                                       | 0/202 [00:00<?, ?it/s]



Processing Training Dataset


 10%|██████                                                       | 16/161 [00:00<00:01, 100.09it/s]


Creating train Tfrecords


100%|█████████████████████████████████████████████████████████████| 41/41 [00:00<00:00, 1397.34it/s]



Creating test Tfrecords
94773248/94765736 [==============================] - 0s 0us/step


# Train

In [5]:
from segmind.keras import CheckpointCallback, KerasCallback

keras_cb = KerasCallback()
ckpt_cb = CheckpointCallback(
            snapshot_interval=10,
            snapshot_path='/tmp',
            checkpoint_prefix='retinanet_test')
pipe.train(
    num_epochs=10,
    snapshot_prefix='test_retinanet',
    snapshot_path='/tmp',
    snapshot_every_n=10,
    batch_size=4,
    #steps_per_epoch=100,
    callbacks=[keras_cb, ckpt_cb])

2020/12/17 08:59:15 INFO segmind.store.tracking.rest_store: Artifact upload HTTP status code: 204


Epoch 1/10
41/40 [==============================] - 30s 722ms/step - loss: 3.1791 - regression_loss: 2.1880 - classification_loss: 0.9911 - val_loss: 1.9959 - val_regression_loss: 1.5388 - val_classification_loss: 0.4571
Epoch 2/10
41/40 [==============================] - 28s 692ms/step - loss: 1.4852 - regression_loss: 1.1122 - classification_loss: 0.3730 - val_loss: 1.1949 - val_regression_loss: 0.9010 - val_classification_loss: 0.2939
Epoch 3/10
41/40 [==============================] - 29s 699ms/step - loss: 1.2042 - regression_loss: 0.9374 - classification_loss: 0.2669 - val_loss: 1.3440 - val_regression_loss: 1.0974 - val_classification_loss: 0.2466
Epoch 4/10
41/40 [==============================] - 28s 691ms/step - loss: 1.0187 - regression_loss: 0.8089 - classification_loss: 0.2098 - val_loss: 0.8803 - val_regression_loss: 0.7127 - val_classification_loss: 0.1676
Epoch 5/10
41/40 [==============================] - 28s 683ms/step - loss: 0.8771 - regression_loss: 0.7344 - classi

2020/12/17 09:05:59 INFO segmind.store.tracking.rest_store: Artifact upload HTTP status code: 204


# Inference

## Imports

In [14]:
from cral.models.object_detection import annotate_image
from PIL import Image
import numpy as np

## Load checkpoint files

In [11]:
checkpoint_file= '/tmp/test_retinanet_final'
prediction_func= pipe.prediction_model(checkpoint_file= checkpoint_file)

converting ...


## Predict and annotate Image

In [16]:
img_path= 'chess_pieces/test/5a35ba2ec3e0d0b2b12b1758a8ac29aa_jpg.rf.280f9940defacbb5d840aef65a9257e5.jpg'

# make prediction
bboxes, scores, labels= prediction_func(img_path)

# draw predictions on image and save
image_array= np.array(Image.open(img_path))
drawn_image= annotate_image(image_array, bboxes, scores, labels, threshold=0.5)
drawn_image.save('image_retinanet.jpeg')